# The No U-Turn Sampler

Hoffman & Gelman 2011: The No U-Turn Sampler
* implement algorithms 1 (basic HMC) & 6 (NUTS with dual averaging)
* test that your implementations work by replicating the following tests in Section 4.1:
 * 250-dimensional Gaussian
 * Bayesian logistic regression & hierarchical Bayesian logistic regression on the UCI german credit data
 * compare the results given by basic HMC & NUTS
 * _compare your results with the same models run in Stan using the provided NUTS sampler_

# Compare your results with the same models run in Stan using the provided NUTS sampler

The number of chains is less than recommended 4 due to slow laptop.

## Download  German credit dataset
There are 24 predictors (x). Predictors are normalized to have a zero mean and an unit variance.

Outcome is whether a person should be refused to get a credit or approced to get one. Values for the outcome are -1 and 1, after recoding 0 and 1.

In [1]:
#%reset
import autograd.numpy as np
import autograd.numpy.random as npr
# Download data and standardize X
# Should have values 1 or -1
import pandas as pd
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import pystan
%matplotlib inline
import HMC
import noUturnSampler
import ChainMix
import Neff
import LikelihoodFunctions
#import CovergenceTests
import importlib
importlib.reload(HMC)
importlib.reload(ChainMix)
importlib.reload(noUturnSampler)
importlib.reload(Neff)
importlib.reload(LikelihoodFunctions)

npr.seed(42)
np.seterr('warn')

credit0= pd.read_csv('dataGerman.tab', delim_whitespace=True)

creditY=np.array(credit0['CREDITRATING'])
creditY[creditY==-1]=0

creditX0=np.array(credit0.loc[:, credit0.columns != 'CREDITRATING'])
creditX=(creditX0 - np.mean(creditX0, axis=0))/np.std(creditX0, axis=0)

# Solve beta values for comparison
logistic = LogisticRegression(fit_intercept=False)
betaCoefficients=logistic.fit(creditX,creditY).coef_


## Bayesian logistic regression using German credit data
Target distribution is the posterior of a Bayesian logistic regression model fit to the German credit dataset.

There are 24-beta coefficients associated with the 24 predictors, and intercept alpha. Coefficients and the intercept are given normal priors with variance of 100.

In [13]:
# Bayesian logistic regression
blrModel = """
    data {
      int<lower=0> N;
      int<lower=0> K;   // number of predictors      
      matrix[N,K] x;
      int<lower=0,upper=1> y[N];
    }
    parameters {
      real alpha;
      vector[K] beta;
    }
    model {
      alpha~normal(0,100);
      beta~normal(0,100);  
      y ~ bernoulli_logit(alpha + x*beta);
    }
"""

credit_dat = {'x': creditX,
                'y': creditY,
                'N': len(creditY),
                'K': creditX.shape[1]
             }

smBlr = pystan.StanModel(model_code=blrModel)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_3e737708637464be719ee19b1a2c8195 NOW.


In [15]:
fitBlrNuts = smBlr.sampling(data=credit_dat, iter=800, chains=3, algorithm='NUTS', control=dict(max_treedepth=15))

In [16]:
print(fitBlrNuts)

Inference for Stan model: anon_model_3e737708637464be719ee19b1a2c8195.
3 chains, each with iter=800; warmup=400; thin=1; 
post-warmup draws per chain=400, total post-warmup draws=1200.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha     -1.22  2.2e-3   0.09  -1.41  -1.28  -1.22  -1.16  -1.05   1790    1.0
beta[1]   -0.76  1.8e-3   0.09  -0.93  -0.82  -0.76  -0.69  -0.58   2322    1.0
beta[2]    0.44  2.7e-3   0.11   0.22   0.37   0.44   0.51   0.66   1591    1.0
beta[3]   -0.41  2.6e-3   0.11  -0.62  -0.48  -0.41  -0.34  -0.19   1749    1.0
beta[4]    0.15  2.9e-3   0.11  -0.07   0.08   0.15   0.23   0.36   1430    1.0
beta[5]   -0.38  2.1e-3    0.1  -0.58  -0.45  -0.38  -0.32  -0.19   2000    1.0
beta[6]   -0.18  2.2e-3   0.09  -0.37  -0.24  -0.18  -0.13 7.3e-4   1716    1.0
beta[7]   -0.16  1.7e-3   0.08  -0.31  -0.21  -0.16   -0.1 1.5e-3   2099    1.0
beta[8]  7.5e-3  2.3e-3   0.09  -0.17  -0.05   0.01   0.07   0.19   1581    1.0
beta[9]    0.1

## Hierarchical bayesian logistic regression
Same beta and alpha coefficients as before but now also the variance parameter is given an exponential prior.

Furthermore interaction effects are added to the model which results to 300-dimensional vector of predictors and 300-dimensional vector of coefficients.

In [5]:
# Create new x-vector for the hierarchical model
from sklearn.preprocessing import PolynomialFeatures
hCreditX=PolynomialFeatures(2, interaction_only=True, include_bias=False).fit_transform(creditX)
print(hCreditX.shape, creditX.shape)
print(creditX[:2,])
print(hCreditX[:2,:30])

(1000, 300) (1000, 24)
[[-1.25456565 -1.23647786  1.38728734 -0.73343195  1.83316907  1.33807849
   0.44932648  1.04698668 -1.29372298  2.76645648  0.46083068  1.02707891
  -0.42828957  1.21459768 -0.19601428 -0.55270519 -0.33886163  0.32021217
  -0.20676767 -0.4669334   0.63444822 -0.14998296 -0.5         0.76635604]
 [-0.45902624  2.24819436 -0.74263181  0.96637654 -0.69970702 -0.31795924
  -0.96364986 -0.76597727 -1.29372298 -1.19140394  0.46083068 -0.704926
  -0.42828957 -0.82331789 -0.19601428 -0.55270519 -0.33886163  0.32021217
  -0.20676767 -0.4669334   0.63444822 -0.14998296 -0.5         0.76635604]]
[[-1.25456565 -1.23647786  1.38728734 -0.73343195  1.83316907  1.33807849
   0.44932648  1.04698668 -1.29372298  2.76645648  0.46083068  1.02707891
  -0.42828957  1.21459768 -0.19601428 -0.55270519 -0.33886163  0.32021217
  -0.20676767 -0.4669334   0.63444822 -0.14998296 -0.5         0.76635604
   1.55124265 -1.74044304  0.92013854 -2.29983094 -1.67870731 -0.56370956]
 [-0.45902624

In [10]:
# Bayesian logistic regression, hierarchical model
blrhModel = """
    data {
      int<lower=0> N;
      int<lower=0> K;   // number of predictors      
      matrix[N,K] x;
      int<lower=0,upper=1> y[N];
    }
    parameters {
      real alpha;
      vector[K] beta;
      real<lower=0> sigma2 ; 
    }
    model {
      sigma2 ~ exponential(0.01);
      alpha~normal(0,sigma2);
      beta~normal(0,sigma2);  
      y ~ bernoulli_logit(alpha + x*beta);
    }
"""

credit_dat = {'x': hCreditX,
                'y': creditY,
                'N': len(creditY),
                'K': hCreditX.shape[1]
             }

smBlrh = pystan.StanModel(model_code=blrhModel)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_540d14dfb012f00e4a5be2b19bd1ab6d NOW.


In [11]:
fitBlrhNuts = smBlrh.sampling(data=credit_dat, iter=700, chains=3, algorithm='NUTS', control=dict(max_treedepth=15))

In [12]:
print(fitBlrhNuts)

Inference for Stan model: anon_model_540d14dfb012f00e4a5be2b19bd1ab6d.
3 chains, each with iter=700; warmup=350; thin=1; 
post-warmup draws per chain=350, total post-warmup draws=1050.

            mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha      -0.42  3.4e-3   0.11  -0.64   -0.5  -0.43  -0.35   -0.2   1099    1.0
beta[1]    -0.57  4.2e-3   0.09  -0.76  -0.63  -0.56   -0.5   -0.4    484    1.0
beta[2]     0.36  2.8e-3    0.1   0.18   0.29   0.36   0.43   0.57   1241    1.0
beta[3]    -0.22  2.4e-3   0.09  -0.41  -0.28  -0.22  -0.17  -0.05   1330    1.0
beta[4]     0.15  2.2e-3   0.11  -0.06   0.08   0.15   0.22   0.37   2425    1.0
beta[5]    -0.27  2.4e-3   0.09  -0.45  -0.32  -0.27  -0.21   -0.1   1281    1.0
beta[6]    -0.21  2.2e-3   0.08  -0.37  -0.26  -0.21  -0.16  -0.06   1288    1.0
beta[7]    -0.07  1.8e-3   0.09  -0.24  -0.13  -0.07  -0.01    0.1   2373    1.0
beta[8]    -0.04  2.4e-3   0.08  -0.21  -0.09  -0.04   0.02   0.13   1241    1.0
beta

In [2]:
# 250-dimensional MVN
mvn250Model = """
    data {
        int wishSize;
        matrix[wishSize,wishSize] A;
        vector[wishSize] mean0;
    }
    parameters {
        vector[wishSize] theta;
    }
    model{
        theta ~ multi_normal(mean0, A);
    }

"""
from scipy.stats import wishart
wishSize=250
precisionMatrixA=wishart.rvs(df=wishSize, scale=np.identity(wishSize), random_state=None)

wishart_data={'A': precisionMatrixA, 'mean0': np.zeros(wishSize), 'wishSize': wishSize}

sm = pystan.StanModel(model_code=mvn250Model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7093332dbae806be2c03b781030a2bc3 NOW.


In [3]:
fitWishartNuts = sm.sampling(data=wishart_data, iter=800, chains=3, algorithm='NUTS')

In [4]:
print(fitWishartNuts)

Inference for Stan model: anon_model_7093332dbae806be2c03b781030a2bc3.
3 chains, each with iter=800; warmup=400; thin=1; 
post-warmup draws per chain=400, total post-warmup draws=1200.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta[1]    -1.67    1.34  15.96 -33.34 -12.44  -1.54   9.28  28.99    141   1.02
theta[2]     -2.9    1.23  16.08 -34.39  -13.5   -2.1   7.39  28.82    172    1.0
theta[3]    -0.47    1.21  15.75 -29.93 -10.76  -0.69   9.67  30.91    169   1.02
theta[4]    -1.92    1.29  16.15 -31.81  -12.8  -2.37   8.87  30.92    156   1.01
theta[5]     2.06    1.12  14.81 -27.25  -7.96   2.12  12.47  29.74    176   1.03
theta[6]     0.43    1.13  15.96 -30.66 -10.14   0.42  11.29  31.27    201    1.0
theta[7]    -0.73    1.09  15.05 -30.37 -10.32  -1.07   9.23  27.42    191   1.02
theta[8]     0.07    1.11  15.42 -31.49  -9.79   0.31  10.48  31.16    193    1.0
theta[9]     0.27    1.48  15.35 -31.26  -9.11   0.94  10.51  29.16    107  